In [ ]:
import os 
import yaml
import datetime
import sys
sys.path.append("..")
from algorithms.utils.wrapper_gym import get_env
from algorithms.offline.any_percent_bc import get_actor_from_checkpoint, eval_actor


In [ ]:
import subprocess
import zipfile

def download_from_gdrive(gdrive_url, output_path):
    try:
        import gdown
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
        import gdown
    gdown.download(gdrive_url, output_path, quiet=False)

GDRIVE_URL = "https://drive.google.com/uc?id=1I6X9Zn0-ha3kINq4i_xn9112jDFPlnlK"
OUTPUT_PATH = "BC-Go2JoystickFlatTerrain.zip"
if not os.path.exists(OUTPUT_PATH):
    download_from_gdrive(GDRIVE_URL, OUTPUT_PATH)

with zipfile.ZipFile(OUTPUT_PATH, 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
CHECKPOINT_PATH = "BC-Go2JoystickFlatTerrain-d7c8b162"
ENV_NAME = "Go2JoystickFlatTerrain"

N_EPISODES = 20
RENDER = True
SEED = 0

In [ ]:
with open(os.path.join(CHECKPOINT_PATH, "config.yaml")) as f:
    config = yaml.safe_load(f)

In [ ]:
render_trajectory = []
def render_callback(_, state):
    render_trajectory.append(state)

env_wrapped = get_env(ENV_NAME, "cuda", render_callback)

state_dim = env_wrapped.observation_space.shape[0] 
action_dim = env_wrapped.action_space.shape[0]
max_action = 1.0

print(state_dim, action_dim, max_action)

In [ ]:
actor = get_actor_from_checkpoint(checkpoint_path=CHECKPOINT_PATH, state_dim=state_dim, action_dim=action_dim, max_action=max_action)

In [ ]:
episode_rewards = eval_actor(actor=actor, env=env_wrapped, device=config["device"], n_episodes=N_EPISODES, seed=SEED, render=RENDER)
print("Mean: ", episode_rewards.mean())
print("Std: ", episode_rewards.std())

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
env_wrapped.save_video(render_trajectory, save_path=f"{CHECKPOINT_PATH}/{ENV_NAME}-{timestamp}.mp4")